In [1]:
%ls newdataset/

interactions_train_1128.csv       whole_data_1128.csv
interactions_validation_1128.csv


In [182]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import json
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import ast

In [184]:
food2vec = pd.read_csv('food2vec_1201.csv')

In [190]:
food2vec.head()

,Ingredient,Vector
0,great northern beans,"[0.1419527, 0.050856147, 0.12818474, 0.2158141..."
1,yellow onion,"[0.08089653, -0.08723446, -0.04058786, 0.33989..."
2,diced green chilies,"[-0.0064202445, 0.18017997, 0.13981035, -0.111..."
3,ground cumin,"[0.038344134, -0.22157283, -0.054828685, 0.113..."
4,garlic powder,"[0.081294596, -0.0376514, -0.010471731, 0.2145..."


In [200]:
food2vec = food2vec.reset_index()

In [201]:
food2vec.head()

,index,Ingredient,Vector
0,0,great northern beans,"[0.1419527, 0.050856147, 0.12818474, 0.2158141..."
1,1,yellow onion,"[0.08089653, -0.08723446, -0.04058786, 0.33989..."
2,2,diced green chilies,"[-0.0064202445, 0.18017997, 0.13981035, -0.111..."
3,3,ground cumin,"[0.038344134, -0.22157283, -0.054828685, 0.113..."
4,4,garlic powder,"[0.081294596, -0.0376514, -0.010471731, 0.2145..."


In [205]:
food2vec_dict = food2vec.set_index(keys='Ingredient')['index'].to_dict()

In [206]:
len(food2vec_dict) [0, 7031]

7032

In [220]:
recipe2vec = pd.read_csv('recipe2vec_1127.csv')

In [221]:
recipe2vec.head()

,i,ingredients,vector
0,0,"['great northern beans', 'yellow onion', 'dice...","[0.04360911116666666, -0.005143107888888889, 0..."
1,1,"[""devil's food cake mix"", 'vegetable oil', 'eg...","[-0.17808505125000001, -0.18320775649999999, 0..."
2,2,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...","[-0.045958727461538465, 0.15744469853846155, 0..."
3,3,"['chicken tenders', 'flour', 'garlic powder', ...","[0.010844033750000003, -0.11082101774999999, -..."
4,4,"['lamb shoulder', 'salt', 'ground black pepper...","[0.026242995312500003, -0.16281941465, 0.00755..."


In [233]:
max_length = len(food2vec_dict)

In [234]:
max_length

7032

In [235]:
indexes = []
recipe2vec = recipe2vec['ingredients']
for each in recipe2vec:
    recipe = ast.literal_eval(each)
    recipe_list = []
    for ingredient in recipe:
        try:
            recipe_list.append(food2vec_dict[ingredient])
        except:
            continue
    recipe_list.extend([max_length]*(20-len(recipe_list)))
    indexes.append(recipe_list)

In [237]:
len(indexes)

178265

In [238]:
recipe2vec = pd.read_csv('recipe2vec_1127.csv')

In [239]:
recipe2vec['indexes'] = indexes

In [240]:
recipe2vec.to_csv('recipe2vec_1203.csv',index=False)

In [241]:
recipe2vec.head()

,i,ingredients,vector,indexes
0,0,"['great northern beans', 'yellow onion', 'dice...","[0.04360911116666666, -0.005143107888888889, 0...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 7032, 7032, 7032, ..."
1,1,"[""devil's food cake mix"", 'vegetable oil', 'eg...","[-0.17808505125000001, -0.18320775649999999, 0...","[9, 10, 11, 12, 7032, 7032, 7032, 7032, 7032, ..."
2,2,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...","[-0.045958727461538465, 0.15744469853846155, 0...","[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 2..."
3,3,"['chicken tenders', 'flour', 'garlic powder', ...","[0.010844033750000003, -0.11082101774999999, -...","[26, 27, 4, 28, 29, 30, 10, 31, 32, 33, 34, 35..."
4,4,"['lamb shoulder', 'salt', 'ground black pepper...","[0.026242995312500003, -0.16281941465, 0.00755...","[36, 28, 37, 10, 38, 39, 27, 40, 41, 42, 43, 3..."


In [211]:
ingre_embeddings_list = [ast.literal_eval(each) for each in food2vec['Vector']]

In [228]:
food2vec_tensor = torch.Tensor(ingre_embeddings_list)

In [229]:
food2vec_tensor = torch.cat((food2vec_tensor, torch.zeros(1, 100)),dim=0)

In [230]:
food2vec_tensor.shape

torch.Size([7033, 100])

In [231]:
torch.save(food2vec_tensor, 'Ingredient_embeddings_1202.pt')

In [242]:
torch.load('Ingredient_embeddings_1202.pt').shape

torch.Size([7033, 100])

In [ ]:
recipe2vec = pd.read_csv('recipe2vec_1127.csv')

In [109]:
def remove_(str_):
    return str_.replace('_', ' ')

In [179]:
food2vec['Ingredient'] = [remove_(ingredient) for ingredient in food2vec['Ingredient']]
food_dict = food2vec.set_index(keys='Ingredient')['Vector'].to_dict()

In [181]:
food2vec.to_csv('food2vec_1201.csv',index=False)

In [98]:
recipe2vec = pd.read_csv('recipe2vec_1127.csv')

In [178]:
recipe2vec.to_csv('recipe2vec_1201.csv',index=False)

In [140]:
# recipe2vec['ingredients'].index
# length_list = [len(ast.literal_eval(each)) for each in recipe2vec['ingredients']]
# min(length_list)

RangeIndex(start=0, stop=178265, step=1)

In [173]:
one_tensor = torch.Tensor(np.array(ingre_embeddings_list))

In [175]:
# length, dim = one_tensor.shape
one_tensor.shape

torch.Size([29, 100])

In [168]:
zero_plus = torch.zeros(20-length,dim)

In [169]:
zero_plus.shape

torch.Size([16, 100])

In [170]:
torch.cat((one_tensor, zero_plus),dim=0).shape

torch.Size([20, 100])

In [55]:
from sklearn.decomposition import PCA
import plotly.express as px

train_data = pd.read_csv('newdataset/interactions_train_1128.csv')


recipe2vec = pd.read_csv('recipe2vec_1127.csv')
recipe2vec_array = recipe2vec['vector'].to_numpy()
recipe2vec_ = torch.Tensor([json.loads(each) for each in recipe2vec_array])

recipe_embeddings = nn.Embedding(178265, 100).from_pretrained(recipe2vec_,freeze=True)

pca = PCA(n_components=2)

In [96]:
# train_data.loc[train_data['u']==3]['i']
# train_data.loc[train_data['i']==7]['u']

In [91]:
index = torch.LongTensor(np.array())
one_user = recipe_embeddings(index)
principalComponents = pca.fit_transform(one_user.detach().numpy())
principalDf = pd.DataFrame(data = principalComponents, columns = ['pc_one', 'pc_two'])

In [92]:
principalDf.index

RangeIndex(start=0, stop=8, step=1)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=principalDf['pc_one'], 
    y=principalDf['pc_two'],
    mode='markers',
    marker=dict(size=15,color='green')
))
fig.show()

In [55]:
1-698901/(25076*178262)

0.9998436497334603

<font color="red">IPython extension to reload modules before executing user code</font>

In [102]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories='auto',sparse=False)
onehot.fit_transform([[0],[1],[2],[3]])

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [6]:
recipe2vec = pd.read_csv('recipe2vec_1126.csv')
recipe2vec['i']

0          16791
1          99809
2          84204
3         172842
4          61982
           ...  
102914    177909
102915    111266
102916      9305
102917     38408
102918    152974
Name: i, Length: 102919, dtype: int64

In [45]:
training_data_path = 'dataset/interactions_train.csv'
validation_data_path = 'dataset/interactions_validation.csv'
test_data_path = 'dataset/interactions_test.csv'
PP_recipes_path = 'dataset/PP_recipes.csv'
RAW_interactions_path = 'dataset/RAW_interactions.csv'
ingredients_map_path = 'dataset/ingredients_map.csv'
RAW_recipes_path = 'dataset/RAW_recipes.csv'

In [46]:
train = pd.read_csv(training_data_path)
validation = pd.read_csv(validation_data_path)

In [54]:
train.index

RangeIndex(start=0, stop=698901, step=1)

In [55]:
validation.index

RangeIndex(start=0, stop=7023, step=1)

In [56]:
698901 + 7023

705924

In [57]:
new_train = train.append(validation)

In [58]:
new_train.to_csv('dataset/interactions_train_1127.csv',index=False)

In [49]:
test = pd.read_csv(test_data_path)

In [50]:
test['i'].describe() # max      178264.000000

count     12455.000000
mean     115488.123485
std       50448.663212
min         102.000000
25%       76904.000000
50%      127793.000000
75%      160024.000000
max      178264.000000
Name: i, dtype: float64

In [47]:
train['i'].describe() # max      178262

count    698901.000000
mean      87519.330993
std       51290.370370
min           0.000000
25%       42988.000000
50%       87424.000000
75%      131731.000000
max      178262.000000
Name: i, dtype: float64

In [48]:
validation['i'].describe() # max      178263.000000

count      7023.000000
mean     100122.151075
std       52051.115779
min         144.000000
25%       56227.000000
50%      104819.000000
75%      146690.500000
max      178263.000000
Name: i, dtype: float64

In [32]:
pp_recipes_data = pd.read_csv(PP_recipes_path)
pp_recipes_data.index

RangeIndex(start=0, stop=178265, step=1)

In [33]:
RAW_recipes_data = pd.read_csv(RAW_recipes_path)
RAW_recipes_data.index

RangeIndex(start=0, stop=231637, step=1)

In [13]:
RAW_recipes_data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [31]:
one_row = RAW_recipes_data['ingredients'][0]

In [36]:
ast.literal_eval(one_row) 

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt']

In [30]:
import json
json.loads("[1, 2, 3]")

[1, 2, 3]

In [10]:
import json
import ast

In [35]:
all_data_list = []

In [49]:
# int(PP_recipes_data.loc[PP_recipes_data['id']==89890890]['i'])

In [ ]:
for index, i in enumerate(RAW_recipes_data['id']):
    try:
        no_recipe = int(pp_recipes_data.loc[pp_recipes_data['id']==i]['i'])
        recipe = RAW_recipes_data['ingredients'][index]
        all_data_list.append([no_recipe, ast.literal_eval(recipe)])
    except:
        continue

<p>check the number of rows</p>

In [38]:
len(all_data_list)

178265

In [39]:
import csv
def write_csv():
    with open('Ingredients_1127.csv', 'w') as myfile:
        wr = csv.writer(myfile)
        wr.writerow(['i','ingredients'])
        for item in all_data_list:
            wr.writerow(item)

In [18]:
# write_csv()
ingredients_data = pd.read_csv('Ingredients_1127.csv')

In [20]:
ingredients_data.head(10)

,i,ingredients
0,145702,"['winter squash', 'mexican seasoning', 'mixed ..."
1,33090,"['prepared pizza crust', 'sausage patty', 'egg..."
2,44921,"['spreadable cheese with garlic and herbs', 'n..."
3,28162,"['milk', 'vanilla ice cream', 'frozen apple ju..."
4,140021,"['fennel seeds', 'green olives', 'ripe olives'..."
5,90348,"['chocolate sandwich style cookies', 'chocolat..."
6,134358,"['sugar', 'unsalted butter', 'bananas', 'eggs'..."
7,71673,"['vanilla wafers', 'butter', 'powdered sugar',..."
8,158958,"['great northern bean', 'chicken bouillon cube..."
9,118914,"['collard greens', 'brown sugar', 'molasses', ..."


In [5]:
ingredients_data = ingredients_data.sort_values(by=['i'])

In [6]:
ingredients_data.head()

,i,ingredients
173846,0,"['great northern beans', 'yellow onion', 'dice..."
55432,1,"[""devil's food cake mix"", 'vegetable oil', 'eg..."
11482,2,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref..."
90512,3,"['chicken tenders', 'flour', 'garlic powder', ..."
92200,4,"['lamb shoulder', 'salt', 'ground black pepper..."


In [5]:
ingredients = ingredients_data['ingredients']

In [40]:
ast.literal_eval(ingredients_data.loc[ingredients_data['i']==178264]['ingredients'].values[0])

['roasting chickens',
 'lemons',
 'rosemary sprigs',
 'paprika',
 'salt',
 'ground black pepper',
 'garlic cloves',
 'onions',
 'celery ribs']

In [37]:
write_ingredients_txt(ingredients_data)
# There are 0 recipes whose the number of ingredients is less than 3 or larger than 20

There are 0 recipes whose the number of ingredients is less than 3 or larger than 20


In [12]:
# replace the empty space with _
def get_txt(data_list):
    string = ''
    for i in data_list:
        if string != '':
            string = string + ' '+ i.replace(' ', '_')
        else:
            string = i.replace(' ', '_')
    return string

In [36]:
def write_ingredients_txt(data):
    count = 0
    with open('ingredients_1127.txt','w') as train_file:
        for i in data.iloc[:].index:
            one_row = data.loc[data['i']==i]['ingredients'].values[0]
            one_list = ast.literal_eval(one_row)
            length = len(one_list)
            if length < 3 and length > 20:
                count = count + 1
            text =  get_txt(one_list)
            train_file.write(text+'\n')
    print('There are {} recipes whose the number of ingredients is less than 3 or larger than 20'.format(count))

In [7]:
recipe_list = []
def recipe_embedding(ingredients_csv, ingredients_txt):
    food2vector = pd.read_csv(ingredients_csv)
    food2vector_dict = food2vector.set_index('Ingredient')['Vector'].to_dict()
    with open(ingredients_txt,'r') as f:
        line = f.readline()
        while line:
            words = line.split()
            recipe_array = np.array([np.array(json.loads(food2vector_dict[word])) for word in words if word in food2vector_dict.keys()])
            recipe_embedding = np.mean(recipe_array, axis=0).tolist()
        #     print(recipe_embedding)
            recipe_list.append(recipe_embedding)
            line = f.readline()
    print(recipe_list[0])

In [11]:
ingredients_csv = 'food2vec_1127.csv'
ingredients_txt = 'ingredients_1127.txt'
recipe_embedding(ingredients_csv, ingredients_txt)

[0.04360911116666666, -0.005143107888888889, 0.030930276777777772, 0.15099080833333337, 0.12966421400000003, 0.16878439, -0.050376431, 0.04871513944444444, 0.047756277666666666, 0.06970340498666666, -0.08699944088888888, 0.017561258111111113, 0.0738900458888889, 0.031199588666666667, -0.051948275555555555, 0.1338699687777778, -0.08174485233333334, -0.06440234322222221, 0.041474127888888886, -0.06931261122222222, 0.11541264333333334, 0.21478299666666667, -0.11072242999999998, -0.1887571918888889, -0.01426841322222222, -0.1506825988888889, 0.03985811211111111, 0.20752809033333333, -0.15123324100000002, 0.08751435100000002, 0.135921304, -0.07479618965555554, 0.10736602377777778, -0.005669685278888892, -0.015072942333333339, 0.08086043511111113, 0.21761395222222224, -0.052467521833333336, -0.17197639888888888, 0.004928480855555548, 0.014917054111111112, 0.22164525233333335, -0.18934589055555556, -0.04747569211111111, -0.13557304744444443, -0.01539593011111111, -0.000393338444444445, 0.0856

In [21]:
food2vec = pd.read_csv('food2vec_1127.csv')

In [22]:
food2vec.head(10)

,Ingredient,Vector
0,great_northern_beans,"[0.1419527, 0.050856147, 0.12818474, 0.2158141..."
1,yellow_onion,"[0.08089653, -0.08723446, -0.04058786, 0.33989..."
2,diced_green_chilies,"[-0.0064202445, 0.18017997, 0.13981035, -0.111..."
3,ground_cumin,"[0.038344134, -0.22157283, -0.054828685, 0.113..."
4,garlic_powder,"[0.081294596, -0.0376514, -0.010471731, 0.2145..."
5,fat-free_chicken_broth,"[0.12689918, 0.003707604, 0.10743472, 0.179322..."
6,fresh_cilantro_leaves,"[-0.081476256, 0.099624015, -0.17738055, 0.052..."
7,extra_virgin_olive_oil,"[-0.008849304, 0.014419843, -0.013997273, 0.39..."
8,sour_cream,"[0.019840665, -0.04861686, 0.20020878, -0.0403..."
9,devil's_food_cake_mix,"[-0.3262564, -0.31572446, -0.13684636, -0.1846..."


In [16]:
ingredients_list.index

RangeIndex(start=0, stop=102919, step=1)

In [12]:
len(recipe_list)

178265

In [13]:
ingredients_data['vector'] = recipe_list

In [15]:
ingredients_data.to_csv('recipe2vec_1127.csv',index=False)

In [16]:
recipe2vec = pd.read_csv('recipe2vec_1127.csv')

In [17]:
recipe2vec.head(10)

,i,ingredients,vector
0,0,"['great northern beans', 'yellow onion', 'dice...","[0.04360911116666666, -0.005143107888888889, 0..."
1,1,"[""devil's food cake mix"", 'vegetable oil', 'eg...","[-0.17808505125000001, -0.18320775649999999, 0..."
2,2,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...","[-0.045958727461538465, 0.15744469853846155, 0..."
3,3,"['chicken tenders', 'flour', 'garlic powder', ...","[0.010844033750000003, -0.11082101774999999, -..."
4,4,"['lamb shoulder', 'salt', 'ground black pepper...","[0.026242995312500003, -0.16281941465, 0.00755..."
5,5,"['pork chops', 'apple', 'dried apricot', 'cran...","[-0.027189695750000003, -0.16694935141666667, ..."
6,6,"['nectarines', 'ear of corn', 'shallot', 'haba...","[-0.049376330499999996, -0.0383035832125, -0.0..."
7,7,"['butter', 'onion', 'garlic', 'potatoes', 'flo...","[-0.018259710176923072, -0.09238103823076924, ..."
8,8,"['strawberries', 'brown sugar', 'cool whip fre...","[-0.2049909151, -0.2363453425, -0.039119478699..."
9,9,"['oats', 'whole wheat flour', 'fiber one cerea...","[-0.181577497, -0.24964736769230766, -0.081040..."


In [12]:
PP_recipes_data = pd.read_csv(PP_recipes_path)
# PP_recipes_data['id'].describe()

In [23]:
int(PP_recipes_data.loc[PP_recipes_data['id']==RAW_recipes_data['id'][0]]['i'] )

145702

In [17]:
PP_recipes_data.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [4]:
validation_data = pd.read_csv(validation_data_path)
test_data = pd.read_csv(test_data_path)

In [5]:
train_data = pd.read_csv(training_data_path)
PP_recipes_data = pd.read_csv(PP_recipes_path)
RAW_interactions_data = pd.read_csv(RAW_interactions_path)
ingredients_map_data = pd.read_csv(ingredients_map_path)

In [6]:
train_data.head()

,user_id,recipe_id,date,rating,u,i
0,2046,4684,2000-02-25,5.0,22095,44367
1,2046,517,2000-02-25,5.0,22095,87844
2,1773,7435,2000-03-13,5.0,24732,138181
3,1773,278,2000-03-13,4.0,24732,93054
4,2046,3431,2000-04-07,5.0,22095,101723


In [25]:
def tabular_preview(ratings, n=5):
    """Creates a cross-tabular view of users vs movies."""
    
    user_groups = ratings.groupby('u')['i'].count()
    top_users = user_groups.sort_values(ascending=False)[:10]

    movie_groups = ratings.groupby('i')['rating'].count()
    top_movies = movie_groups.sort_values(ascending=False)[:10]

    top = (
        ratings.
        join(top_users, rsuffix='_r', how='inner', on='u').
        join(top_movies, rsuffix='_r', how='inner', on='i'))

    return pd.crosstab(top.u, top.i, top.rating, aggfunc=np.sum)

In [26]:
tabular_preview(train_data)[:8]

i,37047,52334,56425,99787,101819,117899,127080,134610,135961,147374
u,,,,,,,,,,
94,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
130,NaN,2.0,3.0,NaN,NaN,NaN,5.0,3.0,NaN,NaN
164,NaN,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN,5.0
193,NaN,5.0,5.0,NaN,5.0,NaN,NaN,4.0,NaN,NaN
208,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
267,5.0,NaN,NaN,5.0,5.0,5.0,NaN,4.0,NaN,5.0
275,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,5.0,5.0
319,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
ingredients_map_data.head()

,id,ingredients
0,1,hershey's semi-sweet chocolate chips
1,1,hershey's semi-sweet chocolate chips
2,1,hershey's semi-sweet chocolate chips
3,1,hershey's semi-sweet chocolate chips
4,1,hershey's semi-sweet chocolate chips


In [8]:
train_data.head(10)

,user_id,recipe_id,date,rating,u,i
0,2046,4684,2000-02-25,5.0,22095,44367
1,2046,517,2000-02-25,5.0,22095,87844
2,1773,7435,2000-03-13,5.0,24732,138181
3,1773,278,2000-03-13,4.0,24732,93054
4,2046,3431,2000-04-07,5.0,22095,101723
5,2046,13307,2000-05-21,5.0,22095,134551
6,2312,780,2000-09-12,5.0,1674,127175
7,2312,51964,2000-09-26,5.0,1674,151793
8,2312,1232,2000-10-17,4.0,1674,15498
9,2312,4397,2000-10-17,5.0,1674,14380


In [16]:
validation_data[validation_data['i']==178263]

,user_id,recipe_id,date,rating,u,i
4392,417131,241491,2007-09-05,4.0,20523,178263


In [17]:
new_validation_data =  validation_data[validation_data['i']!=178263]

In [20]:
new_validation_data.to_csv('new_interactions_validation.csv',index=False)

In [23]:
new_test_data = test_data.loc[(test_data['i']!=178264) & (test_data['i']!=178263)]

In [25]:
new_test_data.to_csv('dataset/new_interactions_test.csv',index=False)

In [24]:
new_test_data['i'].describe() 

count     12453.000000
mean     115478.041516
std       50446.440396
min         102.000000
25%       76883.000000
50%      127789.000000
75%      160014.000000
max      178260.000000
Name: i, dtype: float64

In [33]:
train_data.loc[train_data['u']==0]

,user_id,recipe_id,date,rating,u,i
5702,38094,16954,2002-06-10,5.0,0,1118
5852,38094,30565,2002-06-14,5.0,0,27680
5870,38094,20500,2002-06-17,5.0,0,32541
5943,38094,16391,2002-06-17,5.0,0,137353
7147,38094,34513,2002-07-18,5.0,0,16428
7158,38094,34509,2002-07-18,5.0,0,28815
7755,38094,18007,2002-08-05,5.0,0,65803
8162,38094,31503,2002-08-13,4.0,0,148743
8291,38094,36381,2002-08-15,4.0,0,129939
8315,38094,36653,2002-08-15,4.0,0,99377


In [57]:
train_data['i'].describe()

count    698901.000000
mean      87519.330993
std       51290.370370
min           0.000000
25%       42988.000000
50%       87424.000000
75%      131731.000000
max      178262.000000
Name: i, dtype: float64

In [17]:
698901/178262

3.9206392837508837

In [14]:
698901/25075

27.872422731804587

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Histogram(x=train_data['u'], name='Users'))
fig.add_trace(go.Histogram(x=train_data['i'], name='Recipes'))
fig.update_traces(opacity=0.75)
fig.update_layout(
    title_text='Training Data', # title of plot
    xaxis_title_text='User/Recipe ID', # xaxis label
    yaxis_title_text='Count'# yaxis label
)
fig.show()

In [14]:
PP_recipes_data.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"
